<a href="https://colab.research.google.com/github/ymuto0302/RW2025/blob/main/pretrained_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## VGG16をそのまま利用する
処理の流れは以下のとおり：
1. VGG16モデルを読み込み，評価モードに設定
1. 画像の読み込み
1. 画像の前処理：画像サイズを VGG16 に合わせ，かつテンソル化
1. 画像を VGG16 に与えて推論
1. 推論結果の出力

実行の際，予め example.jpg を Colab へアップロードしておくこと。


In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# ImageNetのクラス名を取得する関数
def get_imagenet_classes():
    # torchvisionの weights からクラス名を取得
    weights = models.VGG16_Weights.IMAGENET1K_V1
    class_names = weights.meta["categories"]
    return class_names

# VGG16をロード
model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
model.eval()  # 評価モードに設定

# ImageNetクラス名を取得
class_names = get_imagenet_classes()

# 画像の前処理を定義
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 画像分類を実行する関数
def classify_image(image_path):
    # 画像を読み込み
    image = Image.open(image_path).convert('RGB')

    # 前処理を適用
    image_tensor = preprocess(image).unsqueeze(0)  # バッチ次元を追加

    # 推論実行
    with torch.no_grad():
        output = model(image_tensor)

    # 確率に変換
    probabilities = torch.nn.functional.softmax(output[0], dim=0)

    # 上位5つの予測を取得
    top5_prob, top5_indices = torch.topk(probabilities, 5)

    # 結果を表示
    print(f"Top 5 predictions for {image_path}:")
    print("-----------------------------------------------")
    for i in range(5):
        class_idx = top5_indices[i].item()
        class_name = class_names[class_idx]
        confidence = top5_prob[i].item() * 100
        print(f"{i+1}. {class_name}: {confidence:.2f}%")

    return top5_indices, top5_prob

if __name__ == "__main__":
    # 画像ファイルのパスを指定
    image_path = "example.jpg"

    classify_image(image_path)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:58<00:00, 9.44MB/s]


Top 5 predictions for example.jpg:
-----------------------------------------------
1. tiger: 70.91%
2. tiger cat: 27.97%
3. jaguar: 0.47%
4. leopard: 0.15%
5. tabby: 0.14%
